In [79]:
import atlite
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
import os
import numpy as np
import pandas as pd

In [ ]:
# Load hydropower plant and hydrobasin data
location_hydro = gpd.read_file(f'Data_25/hydropower_dams_25.gpkg')
location_hydro.rename(columns={'Latitude': 'lat', 'Longitude': 'lon'}, inplace=True)
location_hydro.rename(columns={'head_example': 'head'}, inplace=True)

laos_hydrobasins = gpd.read_file('Data_hydrobasins/hydrobasins_lvl10/hybas_as_lev10_v1c.shp')
laos_hydrobasins['lat'] = location_hydro.geometry.y
laos_hydrobasins['lon'] = location_hydro.geometry.x


In [67]:
cutout_files = [f"Cutouts_atlite/Laos{i}Y.nc" for i in range(1, 3)]
cutout_files

['Cutouts_atlite/Laos1Y.nc', 'Cutouts_atlite/Laos2Y.nc']

In [ ]:
for file in cutout_files:
    # Load the cutout
    cutout = atlite.Cutout(file)

    # Calculate runoff for the current year
    runoff = cutout.hydro(
        plants=location_hydro,
        hydrobasins=laos_hydrobasins,
        per_unit=True  # Normalize output per unit area
    )
    runoff.isel(time=slice(0, 8760))
    
    runoff_list.append(runoff_limited)

In [ ]:
combined_runoff.isel(time=slice(0, 8760))

In [72]:

def calculate_hourly_average_runoff_by_year(cutout_files, location_hydro, laos_hydrobasins):
    """
    Calculate the hourly average runoff for each plant across 5 years.

    Args:
        cutout_files (list): List of file paths to the atlite cutouts for each year.
        location_hydro (GeoDataFrame): Hydropower plant GeoDataFrame.
        laos_hydrobasins (GeoDataFrame): Hydrobasin GeoDataFrame.

    Returns:
        xarray.DataArray: Hourly average runoff for one year, calculated across all years.
    """
    runoff_list = []

    for file in cutout_files:
        # Load the cutout
        cutout = atlite.Cutout(file)

        # Calculate runoff for the current year
        runoff = cutout.hydro(
            plants=location_hydro,
            hydrobasins=laos_hydrobasins,
            per_unit=True  # Normalize output per unit area
        )
        
        runoff = runoff.isel(time=slice(0, 8760))
        
        runoff_list.append(runoff)

    # Concatenate all years along the time dimension
    combined_runoff = xr.concat(runoff_list, dim="time")

    # Group by the hour of the year and calculate the mean for each group
    hourly_average_runoff = (
        combined_runoff.groupby(combined_runoff["time"].dt.dayofyear)
        .mean(dim="time")
        .rename({"dayofyear": "time"})  # Rename dimension to time for clarity
    )

    return hourly_average_runoff

# File paths to the 5-year cutouts
cutout_files = [f"Cutouts_atlite/Laos{i}Y.nc" for i in range(1, 3)]

# Calculate the hourly average runoff for one year
hourly_average_runoff = calculate_hourly_average_runoff_by_year(cutout_files, location_hydro, laos_hydrobasins)

c:\Users\ls2823\AppData\Local\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


KeyboardInterrupt: 

In [84]:
# Function to delete a specific day
def delete_specific_day(data_array, year, month, day):
    """
    Delete a specific day from an xarray.DataArray.

    Args:
        data_array (xarray.DataArray): The DataArray to filter.
        year (int): The year of the day to delete.
        month (int): The month of the day to delete.
        day (int): The day to delete.

    Returns:
        xarray.DataArray: DataArray with the specific day removed.
    """
    # Create a mask for the time dimension, excluding the specified day
    mask = ~((data_array["time"].dt.year == year) &
             (data_array["time"].dt.month == month) &
             (data_array["time"].dt.day == day))

    # Apply the mask to filter the data
    filtered_data_array = data_array.sel(time=mask)

    return filtered_data_array

In [85]:
cutout_files = [f"Cutouts_atlite/Laos{i}Y.nc" for i in range(1, 3)]

# Calculate runoff for each year and store in a list
runoff_list = []
for file in cutout_files:
    cutout = atlite.Cutout(file)
    runoff = cutout.hydro(
        plants=location_hydro,
        hydrobasins=laos_hydrobasins,
        per_unit=True  # Normalize output per unit area
    )
    if "2" in file:  # Adjust this condition based on your file naming convention
            runoff = delete_specific_day(runoff, year=2020, month=2, day=29)
    runoff = runoff.isel(time=slice(0, 8760))

    runoff_list.append(runoff)



c:\Users\ls2823\AppData\Local\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)
c:\Users\ls2823\AppData\Local\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


In [86]:
runoff_list

[<xarray.DataArray (plant: 103, time: 8760)> Size: 7MB
 array([[ 2931388.99974674,  2944661.10368326,  2966097.54450438, ...,
          2914948.46793497,  3338133.03377895,  3004769.69016771],
        [  372513.06459619,   372476.16519585,   372303.26003874, ...,
           253303.88435036,   253303.88435036,   253303.88435036],
        [14215222.50788276, 21238112.99646783, 15700796.29219983, ...,
          7695288.6706861 ,  7167554.97672791,  7574742.34613562],
        ...,
        [  102411.64208451,   106525.57869073,   110697.45388153, ...,
            44168.93568604,    44168.93568604,    44168.93568604],
        [  136748.13060445,   136748.13060445,   136748.13060445, ...,
            92949.44146272,    92949.44146272,    92949.44146272],
        [  115479.33509936,   114273.10763647,   116442.84842874, ...,
           117143.87101783,   115142.94921154,   116280.88517284]])
 Coordinates:
   * plant    (plant) int64 824B 0 1 2 3 4 5 6 7 8 ... 95 96 97 98 99 100 101 102
   * ti

In [92]:
# Combine runoff data from all years into a single DataFrame
hourly_runoff_df = pd.DataFrame()

for runoff in runoff_list:
    # Convert runoff (xarray.DataArray) to pandas DataFrame
    runoff_df = runoff.to_dataframe(name="runoff").reset_index()  # Flatten xarray to a DataFrame
    hourly_runoff_df = pd.concat([hourly_runoff_df, runoff_df], ignore_index=True)

# Preview the combined DataFrame
hourly_runoff_df

,plant,time,runoff
0,0,2019-01-01 00:00:00,2.931389e+06
1,0,2019-01-01 01:00:00,2.944661e+06
2,0,2019-01-01 02:00:00,2.966098e+06
3,0,2019-01-01 03:00:00,3.067301e+06
4,0,2019-01-01 04:00:00,3.104339e+06
...,...,...,...
1804555,102,2020-12-31 19:00:00,2.977560e+05
1804556,102,2020-12-31 20:00:00,2.977560e+05
1804557,102,2020-12-31 21:00:00,2.982942e+05
1804558,102,2020-12-31 22:00:00,2.982942e+05


In [103]:
# Group by plant and time to calculate the average
averaged_runoff_df = (
    hourly_runoff_df.groupby(["time", "plant"])
    .mean()
    .reset_index()
    .pivot(index="time", columns="plant", values="runoff")
)

# Preview the averaged DataFrame
averaged_runoff_df

plant,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,2.931389e+06,372513.064596,1.421522e+07,44041.500946,154305.070461,76329.419604,372513.064596,8.050073e+05,6.741189e+06,1.841406e+06,...,76124.756680,175141.246088,391988.784562,2.068665e+06,1.068816e+06,1.174065e+07,142806.732137,102411.642085,136748.130604,115479.335099
2019-01-01 01:00:00,2.944661e+06,372476.165196,2.123811e+07,44041.500946,154305.070461,76329.419604,372476.165196,8.218255e+05,6.235311e+06,1.789701e+06,...,77099.890233,172623.307634,387713.128739,2.058312e+06,1.074553e+06,1.150117e+07,108866.438831,106525.578691,136748.130604,114273.107636
2019-01-01 02:00:00,2.966098e+06,372303.260039,1.570080e+07,44041.500946,154305.070461,76329.419604,372303.260039,8.676420e+05,6.049058e+06,1.807453e+06,...,80656.405054,168751.617467,389102.140400,1.978055e+06,1.105767e+06,1.137646e+07,109759.627994,110697.453882,136748.130604,116442.848429
2019-01-01 03:00:00,3.067301e+06,367212.572652,1.421322e+07,44025.351072,154305.070461,76329.419604,367212.572652,1.047880e+06,6.092540e+06,1.869963e+06,...,77099.890233,167662.603942,396699.109450,1.885017e+06,1.093283e+06,1.104762e+07,106843.799675,107483.596116,136748.130604,117180.168031
2019-01-01 04:00:00,3.104339e+06,368393.953145,1.408500e+07,42809.153768,166170.897965,75769.384948,368393.953145,1.135858e+06,6.021482e+06,1.930398e+06,...,76612.323457,167662.603942,388038.015303,1.874510e+06,1.078722e+06,1.062091e+07,347025.532996,104125.342414,137165.151980,134921.560900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,4.643357e+06,481066.859934,1.132391e+07,134863.745052,261340.121684,118674.893810,481066.859934,1.244296e+06,4.707843e+06,2.790787e+06,...,63274.774974,415368.270030,940065.053470,4.123461e+06,2.106584e+06,2.819409e+07,292060.915297,62054.762553,149691.059000,297755.963511
2020-12-31 20:00:00,4.651051e+06,481066.859934,1.188057e+07,130015.105708,262039.735682,118674.893810,481066.859934,1.242770e+06,4.366269e+06,2.786874e+06,...,63274.774974,415368.270030,940753.004869,4.034523e+06,2.107664e+06,2.812188e+07,291393.399000,62054.762553,149691.059000,297755.963511
2020-12-31 21:00:00,4.638944e+06,480721.049620,1.189705e+07,129966.656086,262570.238370,118674.893810,480721.049620,1.245605e+06,4.213407e+06,2.788936e+06,...,63274.774974,415372.532103,940753.004869,3.930342e+06,2.104021e+06,2.805235e+07,290683.188235,60764.592927,148415.180971,298294.157824


In [104]:
# Add a column that cycles through 0–8759 for each year
averaged_runoff_df['hour_of_year'] = averaged_runoff_df.index % 8760
averaged_runoff_df

TypeError: cannot perform __mod__ with this index type: DatetimeArray

In [100]:
# Create a numeric index representing the hour of the year
hour_of_year = averaged_runoff_df.index.to_series().dt.dayofyear * 24 + averaged_runoff_df.index.to_series().dt.hour - 24
hour_of_year

time
2019-01-01 00:00:00       0
2019-01-01 01:00:00       1
2019-01-01 02:00:00       2
2019-01-01 03:00:00       3
2019-01-01 04:00:00       4
                       ... 
2020-12-31 19:00:00    8779
2020-12-31 20:00:00    8780
2020-12-31 21:00:00    8781
2020-12-31 22:00:00    8782
2020-12-31 23:00:00    8783
Name: time, Length: 17520, dtype: int32

In [76]:
# File paths to the 5-year cutouts
cutout_files = [f"Cutouts_atlite/Laos{i}Y.nc" for i in range(1, 3)]

runoff_list = []

for file in cutout_files:
    # Load the cutout
    cutout = atlite.Cutout(file)

    # Calculate runoff for the current year
    runoff = cutout.hydro(
        plants=location_hydro,
        hydrobasins=laos_hydrobasins,
        per_unit=True  # Normalize output per unit area
    )
    if "2" in file:  # Adjust this condition based on your file naming convention
            runoff = delete_specific_day(runoff, year=2020, month=2, day=29)
    runoff = runoff.isel(time=slice(0, 8760))

    runoff_list.append(runoff)
    
# Concatenate all years along the time dimension
combined_runoff = xr.concat(runoff_list, dim="time")
combined_runoff


c:\Users\ls2823\AppData\Local\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)
c:\Users\ls2823\AppData\Local\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


<xarray.DataArray (plant: 103, time: 17520)> Size: 14MB
array([[ 2931388.99974674,  2944661.10368326,  2966097.54450438, ...,
         4638944.44768612,  4643189.20633325,  4612761.8284643 ],
       [  372513.06459619,   372476.16519585,   372303.26003874, ...,
          480721.04961959,   478657.48861182,   480893.95477671],
       [14215222.50788276, 21238112.99646783, 15700796.29219983, ...,
        11897050.6970801 , 11448831.37390236, 12777301.33027568],
       ...,
       [  102411.64208451,   106525.57869073,   110697.45388153, ...,
           60764.59292729,    60764.59292729,    62054.76255291],
       [  136748.13060445,   136748.13060445,   136748.13060445, ...,
          148415.18097083,   148415.18097083,   149691.05900017],
       [  115479.33509936,   114273.10763647,   116442.84842874, ...,
          298294.15782438,   298294.15782438,   296128.5101479 ]])
Coordinates:
  * plant    (plant) int64 824B 0 1 2 3 4 5 6 7 8 ... 95 96 97 98 99 100 101 102
  * time     (time) datetime64[ns] 140kB 2019-01-01 ... 2020-12-31T23:00:00

In [83]:
def calculate_average_runoff_for_year(combined_runoff, target_year=2023, hours_per_year=8760):
    """
    Calculate the average runoff across multiple years and assign it to a specific year.

    Args:
        runoff_list (list): List of xarray.DataArray objects with runoff data.
        target_year (int): The year to assign the average runoff (default: 2023).
        hours_per_year (int): Number of hours in the target year (default: 8760).

    Returns:
        xarray.DataArray: Averaged runoff data with time assigned to the target year.
    """

    # Calculate the average runoff along the time dimension
    average_runoff = combined_runoff.mean(dim="time")

    # Create a new time index for the target year
    time_index = xr.DataArray(
        pd.date_range(start=f"{target_year}-01-01", end=f"{target_year}-12-31 23:00", freq="H"),
        dims=["time"]
    )

    # Assign the new time index to the averaged runoff
    average_runoff = average_runoff.expand_dims(dim={"time": time_index}, axis=0).squeeze()
    average_runoff = average_runoff.transpose("plant", "time")

    
    return average_runoff

# Example Usage
# Calculate the average runoff and assign it to 2023
average_runoff_2023 = calculate_average_runoff_for_year(combined_runoff, target_year=2023)

# # Save the result if needed
# average_runoff_2023.to_netcdf("average_runoff_2023.nc")

# Verify the result
average_runoff_2023


C:\Users\ls2823\AppData\Local\Temp\ipykernel_7416\3699843884.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.date_range(start=f"{target_year}-01-01", end=f"{target_year}-12-31 23:00", freq="H"),


<xarray.DataArray (plant: 103, time: 8760)> Size: 7MB
array([[29453313.28245029, 29453313.28245029, 29453313.28245029, ...,
        29453313.28245029, 29453313.28245029, 29453313.28245029],
       [ 4097541.79066344,  4097541.79066344,  4097541.79066344, ...,
         4097541.79066344,  4097541.79066344,  4097541.79066344],
       [38314423.16692269, 38314423.16692269, 38314423.16692269, ...,
        38314423.16692269, 38314423.16692269, 38314423.16692269],
       ...,
       [  560481.37600397,   560481.37600397,   560481.37600397, ...,
          560481.37600397,   560481.37600397,   560481.37600397],
       [ 1275726.17357415,  1275726.17357415,  1275726.17357415, ...,
         1275726.17357415,  1275726.17357415,  1275726.17357415],
       [ 2493457.57852329,  2493457.57852329,  2493457.57852329, ...,
         2493457.57852329,  2493457.57852329,  2493457.57852329]])
Coordinates:
  * time     (time) datetime64[ns] 70kB 2023-01-01 ... 2023-12-31T23:00:00
  * plant    (plant) int64 824B 0 1 2 3 4 5 6 7 8 ... 95 96 97 98 99 100 101 102

In [77]:

# Group by both day of year and hour of day
grouped_runoff = combined_runoff.groupby(
    combined_runoff["time"].dt.dayofyear * 24 + combined_runoff["time"].dt.hour
)

# Calculate the mean for each hour of the year
hourly_average_runoff = grouped_runoff.mean(dim="time")

# Create a new time index (8760 hours) for the average year
total_hours = 366 * 24  # 8760 hours (non-leap year)
hourly_time_index = xr.DataArray(
    range(total_hours),
    dims=["time"],
    coords={"time": range(total_hours)}
)

# Assign the new time index to the result
hourly_average_runoff = hourly_average_runoff.assign_coords(time=hourly_time_index)

ValueError: cannot add coordinates with new dimensions to a DataArray

In [4]:


# Folder containing the input NetCDF files
folder = "Cutouts_atlite"

# List of input NetCDF files in the folder
input_files = [os.path.join(folder, f"Laos{i}Y.nc") for i in range(1, 6)]

# Load all files as xarray datasets and ensure they align
datasets = [xr.open_dataset(file) for file in input_files]

# Calculate the average across all datasets
average_dataset = xr.concat(datasets, dim="time").mean(dim="time")

# # Save the averaged dataset to a new NetCDF file
# output_file = os.path.join(folder, "Laos5AVG.nc")
# average_dataset.to_netcdf(output_file)

# # Close all input datasets
# for ds in datasets:
#     ds.close()

# print(f"Averaged dataset saved to '{output_file}'")


In [57]:
import atlite

cutout = atlite.Cutout("Cutouts_atlite/Laos2Y.nc")
layout = cutout.uniform_layout()

In [58]:
cutout

<Cutout "Laos2Y">
 x = 100.25 ⟷ 107.50, dx = 0.25
 y = 14.00 ⟷ 22.50, dy = 0.25
 time = 2020-01-01 ⟷ 2021-01-01, dt = h
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

In [59]:
runoff = cutout.hydro(
        plants=location_hydro,
        hydrobasins= laos_hydrobasins,
        per_unit=True                    # Normalize output per unit area
    )

runoff

c:\Users\ls2823\AppData\Local\anaconda3\envs\geoh2_master\Lib\site-packages\scipy\sparse\_base.py:730: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)


<xarray.DataArray (plant: 103, time: 8808)> Size: 7MB
array([[4497436.27361378, 4470351.26892043, 4425681.52287539, ...,
        4516272.50467975, 4510448.57549252, 4505929.74781585],
       [ 253303.88435036,  253303.88435036,  253303.88435036, ...,
         472583.88075205,  472583.88075205,  472583.88075205],
       [9802348.56608111, 9868528.08468129, 9810237.04316356, ...,
        9658578.2083064 , 9712234.20576056, 9838248.997201  ],
       ...,
       [  44168.93568604,   44168.93568604,   44168.93568604, ...,
          59537.28303794,   59537.28303794,   59474.42330166],
       [  92949.44146272,   92949.44146272,   92949.44146272, ...,
         145738.92697863,  145738.92697863,  145738.92697863],
       [ 238049.43123477,  238095.88857517,  241497.55443828, ...,
         292838.30135814,  292745.38667733,  292791.84401773]])
Coordinates:
  * plant    (plant) int64 824B 0 1 2 3 4 5 6 7 8 ... 95 96 97 98 99 100 101 102
  * time     (time) datetime64[ns] 70kB 2020-01-01 ... 2021-01-01T23:00:00

In [70]:
runoff.isel(time=slice(0, 8760)) 

<xarray.DataArray (plant: 103, time: 8760)> Size: 7MB
array([[ 4497436.27361378,  4470351.26892043,  4425681.52287539, ...,
         5477774.1084448 ,  5321121.95426902,  5189356.04024703],
       [  253303.88435036,   253303.88435036,   253303.88435036, ...,
          491739.69494435,   491566.78978724,   491739.69494435],
       [ 9802348.56608111,  9868528.08468129,  9810237.04316356, ...,
        10739949.18482184, 12026270.5007286 , 12436636.20982403],
       ...,
       [   44168.93568604,    44168.93568604,    44168.93568604, ...,
           62054.76255291,    62054.76255291,    62054.76255291],
       [   92949.44146272,    92949.44146272,    92949.44146272, ...,
          151091.43496303,   151091.43496303,   151091.43496303],
       [  238049.43123477,   238095.88857517,   241497.55443828, ...,
          304819.65875034,   303693.12951871,   302543.78682773]])
Coordinates:
  * plant    (plant) int64 824B 0 1 2 3 4 5 6 7 8 ... 95 96 97 98 99 100 101 102
  * time     (time) datetime64[ns] 70kB 2020-01-01 ... 2020-12-30T23:00:00

In [71]:
runoff

<xarray.DataArray (plant: 103, time: 8808)> Size: 7MB
array([[4497436.27361378, 4470351.26892043, 4425681.52287539, ...,
        4516272.50467975, 4510448.57549252, 4505929.74781585],
       [ 253303.88435036,  253303.88435036,  253303.88435036, ...,
         472583.88075205,  472583.88075205,  472583.88075205],
       [9802348.56608111, 9868528.08468129, 9810237.04316356, ...,
        9658578.2083064 , 9712234.20576056, 9838248.997201  ],
       ...,
       [  44168.93568604,   44168.93568604,   44168.93568604, ...,
          59537.28303794,   59537.28303794,   59474.42330166],
       [  92949.44146272,   92949.44146272,   92949.44146272, ...,
         145738.92697863,  145738.92697863,  145738.92697863],
       [ 238049.43123477,  238095.88857517,  241497.55443828, ...,
         292838.30135814,  292745.38667733,  292791.84401773]])
Coordinates:
  * plant    (plant) int64 824B 0 1 2 3 4 5 6 7 8 ... 95 96 97 98 99 100 101 102
  * time     (time) datetime64[ns] 70kB 2020-01-01 ... 2021-01-01T23:00:00

### Plot

In [12]:
def hydropower_potential(eta,flowrate,head):
    '''
    Calculate hydropower potential in Megawatts

    Parameters
    ----------
    eta : float
        Efficiency of the hydropower plant.
    flowrate : float
        Flowrate calculated with runoff multiplied by the hydro-basin area, in cubic meters per hour.
    head : float
        Height difference at the hydropower site, in meters.

    Returns
    -------
    float
        Hydropower potential in Megawatts (MW).
    '''
    rho = 997 # kg/m3; Density of water
    g = physical_constants['standard acceleration of gravity'][0] # m/s2; Based on the CODATA constants 2018
    Q = flowrate / 3600 # transform flowrate per h into flowrate per second
    return (eta * rho * g * Q * head) / (1000 * 1000) # MW

def hydropower_potential_with_capacity(flowrate, head, capacity, eta):
    '''
    Calculate the hydropower potential considering the capacity limit

    Parameters
    ----------
    flowrate : float
        Flowrate calculated with runoff multiplied by the hydro-basin area, in cubic meters per hour.
    head : float
        Height difference at the hydropower site, in meters.
    capacity : float
        Maximum hydropower capacity in Megawatts (MW).
    eta : float
        Efficiency of the hydropower plant.

    Returns
    -------
    xarray DataArray
        Capacity factor, which is the limited potential divided by the capacity.
    '''
    potential = hydropower_potential(flowrate, head, eta)
    limited_potential = xr.where(potential > capacity, capacity, potential)
    capacity_factor = limited_potential / capacity
    return capacity_factor

In [ ]:
## Have to create capacity factors

In [13]:
datasets

[<xarray.Dataset> Size: 590MB
 Dimensions:               (x: 30, y: 35, time: 8784)
 Coordinates:
   * x                     (x) float64 240B 100.2 100.5 100.8 ... 107.2 107.5
   * y                     (y) float64 280B 14.0 14.25 14.5 ... 22.0 22.25 22.5
   * time                  (time) datetime64[ns] 70kB 2019-01-01 ... 2020-01-0...
     lon                   (x) float64 240B ...
     lat                   (y) float64 280B ...
 Data variables: (12/15)
     height                (y, x) float32 4kB ...
     wnd100m               (time, y, x) float32 37MB ...
     wnd_shear_exp         (time, y, x) float32 37MB ...
     wnd_azimuth           (time, y, x) float32 37MB ...
     roughness             (time, y, x) float32 37MB ...
     influx_toa            (time, y, x) float32 37MB ...
     ...                    ...
     solar_altitude        (time, y, x) float64 74MB ...
     solar_azimuth         (time, y, x) float64 74MB ...
     temperature           (time, y, x) float32 37MB ...
   

### Create one plot of the average capacity factor over each year, including actual values